In [ ]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/data_movies.csv')

Mounted at /content/drive


In [ ]:
data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
data.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [ ]:
features = ['genres','keywords','tagline','cast','director','crew','title']

for feature in features:
  data[feature] = data[feature].fillna('')

In [ ]:
data['combine'] = data['genres']+' '+data['keywords']+' '+data['tagline']+' '+data['cast']+' '+data['director']+' '+data['crew']+' '+data['title']
data['combine'].head()

,combine
0,Action Adventure Fantasy Science Fiction cultu...
1,Adventure Fantasy Action ocean drug abuse exot...
2,Action Adventure Crime spy based on novel secr...
3,Action Crime Drama Thriller dc comics crime fi...
4,Action Adventure Science Fiction based on nove...


In [ ]:
vector = TfidfVectorizer()
f_vector = vector.fit_transform(data['combine'])
print(f_vector)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 780832 stored elements and shape (4803, 228234)>
  Coords	Values
  (0, 182789)	0.004930194197293197
  (0, 182917)	0.005655051945673446
  (0, 195923)	0.006972798011765814
  (0, 218526)	0.006501734206339019
  (0, 196336)	0.006497948542530907
  (0, 191995)	0.013427780727704678
  (0, 190579)	0.013940105137796054
  (0, 197350)	0.010263335757894692
  (0, 220491)	0.021036981519219976
  (0, 226233)	0.007796985119589528
  (0, 190969)	0.015667389080321448
  (0, 220226)	0.013212994953562873
  (0, 195281)	0.014254353534308371
  (0, 222728)	0.003461824855584277
  (0, 227254)	0.007869799266090704
  (0, 212405)	0.007585069498333175
  (0, 213164)	0.01649396188692514
  (0, 217732)	0.015185914090426642
  (0, 227275)	0.01409118012133932
  (0, 228116)	0.01093969030690798
  (0, 217650)	0.013427780727704678
  (0, 219684)	0.012928079140450292
  (0, 226400)	0.010810213187977051
  (0, 221100)	0.018227524443084644
  (0, 205742)	0.011749621556812327
 

In [ ]:
similarity = cosine_similarity(f_vector)
print(similarity)

[[1.         0.5708211  0.8250532  ... 0.35706057 0.08242202 0.1987476 ]
 [0.5708211  1.         0.57647053 ... 0.32139219 0.07066045 0.17332541]
 [0.8250532  0.57647053 1.         ... 0.37433542 0.09198551 0.20208881]
 ...
 [0.35706057 0.32139219 0.37433542 ... 1.         0.04837375 0.1352595 ]
 [0.08242202 0.07066045 0.09198551 ... 0.04837375 1.         0.02853306]
 [0.1987476  0.17332541 0.20208881 ... 0.1352595  0.02853306 1.        ]]


In [ ]:
movie_name = input('Enter your movie name : ')

Enter your movie name : Captain


In [ ]:
every_title = data['title'].tolist()
print(every_title)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [ ]:
find_close_match = difflib.get_close_matches(movie_name, every_title)
print(find_close_match)

['Captive', 'Catwoman', 'Adaptation.']


In [ ]:
closest_match = find_close_match[0]
print(closest_match)

Captive


In [ ]:
index_of_movie = data[data.title == closest_match]['index'].values[0]
print(index_of_movie)

4100


In [ ]:
data['title'][index_of_movie]

'Captive'

In [ ]:
similar_score = list(enumerate(similarity[index_of_movie]))
print(similar_score)

[(0, np.float64(0.6288427324156718)), (1, np.float64(0.5132974665197069)), (2, np.float64(0.6552427150808222)), (3, np.float64(0.6688384571369002)), (4, np.float64(0.6533555002364063)), (5, np.float64(0.5591344363630553)), (6, np.float64(0.5387500835181743)), (7, np.float64(0.6074380452570148)), (8, np.float64(0.47820345291328736)), (9, np.float64(0.6575401694990152)), (10, np.float64(0.44696954399969224)), (11, np.float64(0.40364385487178617)), (12, np.float64(0.4647277737985704)), (13, np.float64(0.5120167688437557)), (14, np.float64(0.5728333733279815)), (15, np.float64(0.6147938799149268)), (16, np.float64(0.6342382885937198)), (17, np.float64(0.5432750463322602)), (18, np.float64(0.44841959148554905)), (19, np.float64(0.6441982637714943)), (20, np.float64(0.6343321286778272)), (21, np.float64(0.4098897221374948)), (22, np.float64(0.6443080374812062)), (23, np.float64(0.5017121354653701)), (24, np.float64(0.5255254900481647)), (25, np.float64(0.5947561699624252)), (26, np.float64(0

In [ ]:
similar_score_sort = sorted(similar_score, key = lambda x:x[1], reverse = True)
print(similar_score_sort)

[(4100, np.float64(0.9999999999999998)), (223, np.float64(0.6773603637648187)), (186, np.float64(0.6736069126777754)), (112, np.float64(0.6735879035031738)), (303, np.float64(0.6733296525910338)), (1430, np.float64(0.6732711796326237)), (182, np.float64(0.6726944741202855)), (412, np.float64(0.672402856171847)), (250, np.float64(0.671880864898802)), (185, np.float64(0.671158294082356)), (365, np.float64(0.6703590894025347)), (171, np.float64(0.6693534607173681)), (212, np.float64(0.6689551516226188)), (338, np.float64(0.6689242886383812)), (3, np.float64(0.6688384571369002)), (275, np.float64(0.6687274041881807)), (408, np.float64(0.6678600354404055)), (320, np.float64(0.6673121657849682)), (828, np.float64(0.6658522571326233)), (28, np.float64(0.6657506531378118)), (165, np.float64(0.6653650967963602)), (654, np.float64(0.6647810666144842)), (673, np.float64(0.6645643454595124)), (93, np.float64(0.6643696947806046)), (191, np.float64(0.6640588548673512)), (714, np.float64(0.6639281638

In [ ]:
for i in range(30):
  index = similar_score_sort[i][0]
  print(data['title'][index])

Captive
The Chronicles of Riddick
Bad Boys II
Alexander
Catwoman
Cursed
Ant-Man
The Core
The Aviator
War of the Worlds
Contact
Master and Commander: The Far Side of the World
The Day After Tomorrow
The Alamo
The Dark Knight Rises
Minority Report
Collateral Damage
Black Hawk Down
Kill Bill: Vol. 1
Jurassic World
Hulk
Blade: Trinity
Austin Powers in Goldmember
Terminator 3: Rise of the Machines
Harry Potter and the Prisoner of Azkaban
Collateral
Blade
Tears of the Sun
Blade II
The League of Extraordinary Gentlemen


In [ ]:
joblib.dump(data, 'data.pkl')
joblib.dump(similarity, 'similarity.pkl')
joblib.dump(vector, 'tfidf.pkl')

['tfidf.pkl']